In [31]:
import os
from os.path import basename
import pandas as pd
from pandas import *
import time

In [32]:
#  Read in the csv files for Parcel ID and Taz and Parcel ID and Use Type for join
parcels = pd.read_csv(r'D:\soundcast\soundcast\inputs\buffered_parcels.dat', sep=' ')

In [33]:
# Ideally, we attach a land use code to the urbansim input and filter directly from there
# For now, we need to join in parcel land use info
parcel_lu = pd.read_csv(r'R:\Craig\Trucks\inputs\TripGen\Base\parcels\parcels_allowable_lu.txt')

In [34]:
df = parcels.merge(parcel_lu, left_on='parcelid',right_on='parcel_id')

In [35]:
# List of allowable truck land uses
truck_uses = ['Agriculture','Fisheries','Forest, harvestable','Forest, protected','Industrial','Military','Mining','Warehousing']

In [36]:
# select truck rows only, using the allowable truck land uses
truck_df = df[df["generic_land_use_1"].isin(truck_uses)]

In [37]:
# For future reference, lookup all the use_codes for these generic_land_uses
# return list of tuples that includes the generic name and all the use codes
truck_use_codes = np.array(truck_df.groupby(['generic_land_use_1','use_code']).count().index)    # returns list of tuples
truck_use_codes = [i[1] for i in truck_use_codes]    # extract only the use_code field, drop the generic_land_use_1 data

# in future, probably want to use the use_code field to select truck rows like this:
# df[df['use_code'].isin(truck_use_codes)]

In [38]:
# Add a flag for truck allowable field
truck_df['trucks_allowed_parcel'] = 1

C:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [39]:
# merge the truck_df back into the main df
df = df.merge(truck_df[['parcelid','trucks_allowed_parcel']], how='left')
df['trucks_allowed_parcel'].fillna(0,inplace=True)    # Truck restricted parcels get a 0

In [40]:
# Now, groupby TAZ and create new flag that allows trucks on TAZ with allowable land use
df_taz = pd.DataFrame(df.groupby('taz_p').sum()[['trucks_allowed_parcel']])
df_taz['trucks_allowed_taz'] = pd.cut(df_taz['trucks_allowed_parcel'], bins=[0,1,df_taz['trucks_allowed_parcel'].max()], labels=[0,1], include_lowest=True)

In [41]:
df_taz['trucks_allowed_taz'].astype('int').sum()

1860

In [42]:
import h5py
import pandas as pd

In [43]:
# look at the output for midday

newtruck = h5py.File(r'D:\soundcast\soundcast\inputs\4k\auto.h5')

In [44]:
oldtruck = h5py.File(r'R:\SoundCast\releases\TransportationFutures2010\inputs\4k\auto.h5')

In [45]:
# Get results by regional growth center
taz_lookup = pd.read_csv(r'R:\SoundCast\releases\TransportationFutures2010\scripts\summarize\TAZ_TAD_County.csv')

In [46]:
h5file = newtruck

In [47]:
def prod_results(h5file):    
    production_total = {}
    for trk in ['lttrk', 'mdtrk','hvtrk']:
        df = 0
        for tod in ['am','md','pm','ev','ni']:
            df += pd.DataFrame(h5file[tod][tod[0]+trk][:])
        df = pd.DataFrame([df.sum(axis=0),df.index]).T
        df.columns = [trk,'TAZ']
        df['TAZ'] = [i+1 for i in df['TAZ']]    # offset TAZ by 1 to get true value

        # Join to geographic data
        df = df.merge(taz_lookup, on='TAZ', how='left')

        production_total[trk] = df
    return production_total

In [48]:
def attr_results(h5file):
    attraction_total = {}
    for trk in ['lttrk', 'mdtrk','hvtrk']:
        df = 0
        for tod in ['am','md','pm','ev','ni']:
            df += pd.DataFrame(h5file[tod][tod[0]+trk][:])
        df = pd.DataFrame([df.sum(axis=1),df.index]).T
        df.columns = [trk,'TAZ']
        df['TAZ'] = [i+1 for i in df['TAZ']]    # offset TAZ by 1 to get true value

        # Join to geographic data
        df = df.merge(taz_lookup, on='TAZ', how='left')

        attraction_total[trk] = df
    return attraction_total

In [49]:
# Format output
pd.options.display.float_format = '{:,.0f}'.format

In [50]:
truck_list = ['lttrk', 'mdtrk','hvtrk']

In [51]:
def dict_to_df(dict_in, geography):
    df = pd.DataFrame([dict_in[trk].groupby(geography).sum()[trk] for trk in truck_list])
    df.loc['Total'] = df.sum()
    return df

In [52]:
def compare_df(df1,df1_name,df2,df2_name):
    '''Compare 2 identical dataframes'''
    df = pd.concat([df1,df2], axis=1)[df1.columns]
    df.columns = pd.MultiIndex.from_product([np.array(df1.columns),[df1_name,df2_name]])
    return df

---

Compare results of the old with the new!

In [53]:
old_prod = prod_results(oldtruck)

In [54]:
old_attr = attr_results(oldtruck)

In [55]:
new_prod = prod_results(newtruck)

In [56]:
new_attr = attr_results(newtruck)

In [57]:
old_pro_sum = dict_to_df(old_prod, 'County')
new_pro_sum = dict_to_df(new_prod, 'County')
compare_df(old_pro_sum,'Old 2010',new_pro_sum,'New 2010').to_clipboard()

In [71]:
tod = 'am'
trk = 'hvtrk'
df = pd.DataFrame(h5file[tod][tod[0]+trk][:])

In [74]:
df.sum().sum()

21927.413814836123

In [67]:
# Get external totals
low_stat = 3733
high_stat = 3750

new_prod['hvtrk'].iloc[3710:high_stat]

,hvtrk,TAZ,TAD,OldDistric,County,District,New DistrictName
3710,0,"3,711",nan,nan,NaN,nan,NaN
3711,"5,657","3,712",nan,nan,NaN,nan,NaN
3712,0,"3,713",nan,nan,NaN,nan,NaN
3713,64,"3,714",nan,nan,NaN,nan,NaN
3714,0,"3,715",nan,nan,NaN,nan,NaN
3715,55,"3,716",nan,nan,NaN,nan,NaN
3716,0,"3,717",nan,nan,NaN,nan,NaN
3717,201,"3,718",nan,nan,NaN,nan,NaN
3718,0,"3,719",nan,nan,NaN,nan,NaN
3719,0,"3,720",nan,nan,NaN,nan,NaN
